# Optimisation of LCA models

* Minimise environment impact and/or cost of a network of processes 
* OpenLCA for lifecycle assessment data
* Pyomo for mathematical optimisation
* Jupyter notebooks for prototyping
* **Qt GUI for configuration and solution**

# OpenLCA

* Java application built by Greendata
* Front-end to life-cycle database of processes and flows
* Ecoinvent system database has ~18000 processes ~7000 flows
* System processes broken down into elementary flows
* Contains methodologies to calculate impact of system
* **Cannot feasibly optimise over multiple alternatives**

# Pyomo

* An algebraic modelling langugage like GAMS, AIMMS
* Implemented as an object model in Python package `pyomo`
* Eases translation of mathematical optimisation problem to actual implementation
* Optimisation problems are formulated by building concrete or **abstract model**


# Mola

Python package to build optimisation models using openLCA data

| Module | Description |
| ------ | ----------- |
| `mola.dataimport` | Convert openLCA Ecoinvent system db to sqlite |
| `mola.dataview` | Build SQL to query sqlite db |
| `mola.specificationv5` | Classes each containing abstract Pyomo model |
| `mola.build` | Configure, populate, solve optimisation problem |
| `mola.output` | Turn pyomo objects into Pandas DataFrames |

## Example

`mola.dataview` is a module of functions to build SQL and a return Pandas DataFrame.

In [ ]:
import mola.dataview as dv
dbconn = dv.get_sqlite_connection()

In [ ]:
dv

In [ ]:
dv.get_processes(dbconn, name=['%lemon%'])

## Specification

Decorates a `pyomo` abstract model so that we can build an interface using introspection
```
class TestSpecification(Specification):
    name = 'Test Specification'
    user_defined_sets = {'A': {'doc': 'test set', 'lookup': False}}
    user_defined_parameters = {'Q': {'index': ['A'], 'doc': 'test parameters'}}

    def __init__():
        self.abstract_model = AbstractModel() # build user defined and db parameters
        
    def populate(json_files): # populates abstract model
        dp = DataPortal()
```

# MolaQT

* Front-end to Mola using `pyqt`
* MolaQT is a data editor and execution environment
* Mainly focused on using openLCA data for optimisation, but can be used more generally
* There is (currently) separation between **model building** (Mola) and **model usage** (MolaQT)
* Use Pyomo to formulate model `Specification` in Jupyter or script



## Abstract Controller

MolaQT has a Controller object for each model. It is a collection widgets that manipulate model configuration data. 

```
class Controller(QWidget):

  def __init__(spec):
      self.spec = spec
      self.sets = spec.get_default_sets()
      self.parameters = spec.get_default_parameters()
      self.lookup = LookupTables(self.db) # if required

  def get_config(self): # get current state
```

## StandardController

Aims to Mirror Pyomo modelling structure so flexible but low-level.

```
class StandardController(Controller):

  def __init__(spec):
       self.sets_editor = SetsEditor(self.sets, self.lookup)
       self.parameters_editor = ParametersEditor(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
  
```

## CustomController

High-level widgets tied to a particular model and workflow. Hardcodes the model to the UI.

```
class CustomController(Controller):

  def __init__(spec):
       self.objectives = ObjectivesWidget(self.sets, self.lookup)
       self.ProcessFlow = ProcessFlow(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
 ```

# Demo of molaqt

* General overview: Github, DB, General Specification, Models
* AIMMS tutorial problem: interface and model construction
* Orange production and transport: openLCA data (StandardController)
* Orange production and transport: openLCA data (CustomController)

# Development work

* Tasks: pyomo.network - MultiTaskController, ScheduleController, DashboardController
* Storage: AIMMS advanced example, Sargent papers, STNs, RTNs
* Expand `output` module to export data for graphing
* More sophistical openLCA models e.g. oil to diesel conversion
* Dynamic model generation